In [1]:
from statistics import mode
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
def gini(y):
    "Return the gini impurity score for values in y"
    classes = np.unique(y)
    cnt_class = dict( zip(classes,np.zeros(len(y))) )
    for c in y:
        cnt_class[c] += 1
    cnt = np.array(list(cnt_class.values()))
    p = cnt/len(y)

    return (1-np.sum(p**2))

In [3]:
class DecisionNode:
    def __init__(self, col, split, lchild, rchild):
        self.col = col
        self.split = split
        self.lchild = lchild
        self.rchild = rchild

    def predict(self, x_test):
        # Make decision based upon x_test[col] and split
        if x_test[self.col] <= self.split:
            return self.lchild
        else: 
            return self.rchild

In [4]:
class LeafNode:
    def __init__(self, y, prediction):
        """
        Create leaf node from y values and prediction; 
        prediction is mean(y) for regression or mode(y) for classification
        """
        self.y = y
        self.n = len(y)
        self.prediction = prediction

    def predict(self, x_test):
        if self.prediction == "regression":
            return np.mean(self.y)
        return mode(self.y)

In [5]:
class DecisionTree_new:
    def __init__(self, min_samples_leaf=1, loss=None):
        self.min_samples_leaf = min_samples_leaf
        self.loss = loss # loss function; either np.std or gini

    def fit(self, X, y):
        """
        Create a decision tree fit to (X,y) and save as self.root, the root of
        our decision tree, for either a classifier or regressor.
        This function is a wrapper around fit_() that just stores the tree in self.root.
        """
        self.root = self.fit_(X, y)

    def bestsplit(self, X, y):

        best = (-1, -1, self.loss(y))

        for col in range(X.shape[1]): 
            if len(X) < 50:
                idx = np.random.choice(X.shape[0],size=len(X),replace=False)
            else:
                idx = np.random.choice(X.shape[0],size=11,replace=False)
            x = X[:,col][idx]
            y_new = y[idx]
            for split in x:
                yl = y_new[x<=split]
                yr = y_new[x>split]
                if len(yl)<self.min_samples_leaf or len(yr)<self.min_samples_leaf:
                    continue
                l = ( len(yl)*self.loss(yl) + len(yr)*self.loss(yr) )/len(y_new)
                #weighted average of subregion losses
                if l == 0:
                    return (col,split)
                if l < best[2]:
                    best = (col, split, l)

        return (best[0],best[1])
    
    def fit_(self, X, y):
        """
        Recursively create and return a decision tree fit to (X,y).
        This function invokes either RegressionTree621.create_leaf() or ClassifierTree621.create_leaf() 
        to create the appropriate leaf node.
        """
        if X.shape[0] <= self.min_samples_leaf:
            return self.create_leaf(y)
        
        col,split = self.bestsplit(X,y)

        if col==-1:
            return self.create_leaf(y)

        lchild = self.fit_(X[X[:,col]<=split],y[X[:,col]<=split])
        rchild = self.fit_(X[X[:,col]>split],y[X[:,col]>split])

        return DecisionNode(col,split,lchild,rchild)

        
    def predict(self, X_test):
        """
        Make a prediction for each record in X_test and return as array.
        This method is inherited by RegressionTree_new and ClassifierTree_new and
        works for both without modification.
        """
        def pred(node,x_test):
            if isinstance(node,LeafNode):
                return node.predict(x_test)
            else:
                return pred(node.predict(x_test),x_test)

        preds = np.array(list(map(lambda x_test: pred(self.root,x_test), X_test)))

        return preds  

In [6]:
class RegressionTree_new(DecisionTree_new):
    def __init__(self, min_samples_leaf=1):
        super().__init__(min_samples_leaf, loss=np.std)

    def score(self, X_test, y_test):
        "Return the R^2 of y_test vs predictions for each record in X_test"
        preds = self.predict(X_test)
        r2 = r2_score(y_test,preds)
        return r2
        
    def create_leaf(self, y):
        """
        Return a new LeafNode for regression.
        """
        return LeafNode(y,'regression')

In [7]:
class ClassifierTree_new(DecisionTree_new):
    def __init__(self, min_samples_leaf=1):
        super().__init__(min_samples_leaf, loss=gini)

    def score(self, X_test, y_test):
        "Return the accuracy_score() of y_test vs predictions for each record in X_test"
        preds = self.predict(X_test)
        acc = accuracy_score(y_test,preds)
        return acc
        
    def create_leaf(self, y):
        """
        Return a new LeafNode for classification.
        """
        return LeafNode(y,'classification')